In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel
import pyspark.sql 
from pyspark.sql import SQLContext
from pyspark.sql.functions import * 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,FloatType
from pyspark.sql.types import Row


In [ ]:
spark = SparkContext(master="local", appName="clase14")
sqlContext = SQLContext(spark)

In [ ]:
!head -n 5 /home/spark/curso-apache-spark-platzi/proyecto/athlete_events.csv

In [ ]:
def dropFirstRow(index,iterator):
     return iter(list(iterator)[1:]) 



In [ ]:
path = "/home/spark/Downloads/curso-apache-spark-platzi-master/files/"

deportistaOlimpicoRDD =  spark.textFile(path+"deportista.csv").map(lambda line : line.split(","))
deportistaOlimpico2RDD = spark.textFile(path+"deportista2.csv").map(lambda line : line.split(","))
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpico2RDD)

deportistaOlimpicoRDD=deportistaOlimpicoRDD.mapPartitionsWithIndex(dropFirstRow)

deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l : (
int(l[0]),
l[1],
int(l[2]),
int(l[3]),
int(l[4]),
float(l[5]),
int(l[6])
))

schema = StructType([
StructField("deportista_id",IntegerType(),False)     ,
StructField("nombre",StringType(),False)        ,
StructField("genero",IntegerType(),False)        ,
StructField("edad",IntegerType(),True)      ,
StructField("altura",IntegerType(),True)        ,
StructField("peso",FloatType(),True)      ,
StructField("equipo_id",IntegerType(),True)     
])

deportistaOlimpicoDF = sqlContext.createDataFrame(deportistaOlimpicoRDD,schema)

In [ ]:
deportistaOlimpicoRDD =  spark.textFile(path+"deportistaError.csv").map(lambda line : line.split(","))
deportistaOlimpicoRDD=deportistaOlimpicoRDD.mapPartitionsWithIndex(dropFirstRow)

deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l : (
l[0],
l[1],
l[2],
l[3],
l[4],
l[5],
l[6]
))

schema = StructType([
StructField("deportista_id",StringType(),False)     ,
StructField("nombre",StringType(),False)        ,
StructField("genero",StringType(),False)        ,
StructField("edad",StringType(),True)      ,
StructField("altura",StringType(),True)        ,
StructField("peso",StringType(),True)      ,
StructField("equipo_id",StringType(),True)     
])

deportistaError = sqlContext.createDataFrame(deportistaOlimpicoRDD,schema)

In [ ]:
deportistaError.show()

In [ ]:
def ci(value: str) -> int:
    return int(value) if len(value) > 0 else None
ci_udf = udf(lambda z : ci(z), IntegerType())
sqlContext.udf.register("ci_udf", ci_udf)

deportistaError.select(ci_udf("altura").alias("altura")).show()

In [ ]:
deportistaError.show()

In [ ]:
paisesRDD = spark.textFile(path+"paises.csv").map(lambda line : line.split(","))
paisesRDD = paisesRDD.mapPartitionsWithIndex(dropFirstRow)

paisesRDD = paisesRDD.map(lambda l : (
int(l[0]),
l[1],
l[2]
))

schema = StructType([
StructField("id",IntegerType(),False),
StructField("equipo",StringType(),False),
StructField("sigla",StringType(),False)
])

paisesDF = sqlContext.createDataFrame(paisesRDD,schema)

In [ ]:
deportesOlimpicosRDDSchema = StructType(
    [StructField("deporte_id",IntegerType(),False),
     StructField("deporte",StringType(),False)
    ])

deportesDF = sqlContext.read.schema(deportesOlimpicosRDDSchema).option("header","true").csv(path+"deporte.csv")

In [ ]:
eventoSchema= StructType([
    StructField("evento_id",IntegerType(),False),
    StructField("nombre",StringType(),False),
    StructField("deporte_id",IntegerType(),False)
])

deportesOlimpicosDF = sqlContext.read.schema(eventoSchema).option("header","true").csv(path+"evento.csv")

In [ ]:

juegoSchema = StructType([
    StructField("juego_id",IntegerType(),False),
    StructField("anio",StringType(),False),
    StructField("temporada",StringType(),False),
    StructField("ciudad",StringType(),False),
])
juegoDF = sqlContext.read.schema(juegoSchema).option("header","true").csv(path+"juegos.csv")

resultadoSchema = StructType([
    StructField("resultado_id",IntegerType(),False),
    StructField("medalla",StringType(),False),
    StructField("deportista_id",IntegerType(),False),
    StructField("juego_id",IntegerType(),False),
    StructField("evento_id",IntegerType(),False),
])
resultadoDF = sqlContext.read.schema(resultadoSchema).option("header","true").csv(path+"resultados.csv")

In [ ]:
deportesDF.take(5)

In [ ]:
deportesOlimpicosDF.take(5)

In [ ]:
paisesDF.take(5)

In [ ]:
juegoDF.take(5)

In [ ]:
deportistaOlimpicoDF.take(5)

In [ ]:
resultadoDF.take(5)

In [ ]:
deportesDF.printSchema()

In [ ]:
deportistaOlimpicoDF.printSchema()

In [27]:
deportistaOlimpicoDF = deportistaOlimpicoDF.withColumnRenamed("genero","sexo").drop("altura")

In [ ]:
deportistaOlimpicoDF.printSchema()

In [ ]:
from pyspark.sql.functions import *
deportistaOlimpicoDF = deportistaOlimpicoDF.select("deportista_id","nombre",
                            col("edad").alias("edadAlJugar"),"equipo_id")

In [ ]:
deportistaOlimpicoDF.show(5)

In [37]:
deportistaOlimpicoDF = deportistaOlimpicoDF.filter( (deportistaOlimpicoDF.edadAlJugar != 0))

In [ ]:
deportistaOlimpicoDF.sort("edadAlJugar").show()

In [55]:
deportistaOlimpicoDF.join(resultadoDF, deportistaOlimpicoDF.deportista_id == resultadoDF.deporitsta_id,"left") \
    .join(juegoDF,juegoDF.juego_id == resultadoDF.juego_id,"left") \
    .join(deportesOlimpicosDF, deportesOlimpicosDF.evento_id == resultadoDF.evento_id,"left") \
    .select(deportistaOlimpicoDF.nombre,col("edad").alias("Edad el jugar"),
           "medalla",col("anio").alias("Año de juego"),
           deportesOlimpicosDF.nombre.alias("Nombre de disciplina")).show()

+--------------------+-------------+-------+-------------+--------------------+
|              nombre|Edad el jugar|medalla| Año de juego|Nombre de disciplina|
+--------------------+-------------+-------+-------------+--------------------+
|           A Dijiang|           24|     NA|  1992 Verano|Basketball Men's ...|
|            A Lamusi|           23|     NA|  2012 Verano|Judo Men's Extra-...|
| Gunnar Nielsen Aaby|           24|     NA|  1920 Verano|Football Men's Fo...|
|Edgar Lindenau Aabye|           34|   Gold|  1900 Verano|Tug-Of-War Men's ...|
|Christine Jacoba ...|           21|     NA|1994 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|           21|     NA|1994 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|           21|     NA|1992 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|           21|     NA|1992 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|           21|     NA|1988 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|           21|    

In [68]:
resultadoDF.filter(resultadoDF.medalla != "NA") \
    .join(deportistaOlimpicoDF,deportistaOlimpicoDF.deportista_id == resultadoDF.deportista_id,"left") \
    .join(paisesDF,paisesDF.id == deportistaOlimpicoDF.equipo_id, "left") \
    .select("medalla", "equipo","sigla").sort( col("sigla").desc() ).show()

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



In [78]:
#######Funciones escalares
paisesDF.printSchema()

root
 |-- id: integer (nullable = false)
 |-- equipo: string (nullable = false)
 |-- sigla: string (nullable = false)



In [87]:
medallistaXAnio = deportistaOlimpicoDF.join(resultadoDF, deportistaOlimpicoDF.deportista_id ==resultadoDF.deportista_id,"left") \
        .join(juegoDF, juegoDF.juego_id == resultadoDF.juego_id,"left") \
        .join(paisesDF,deportistaOlimpicoDF.equipo_id == paisesDF.id,"left") \
        .join(deportesOlimpicosDF, deportesOlimpicosDF.evento_id == resultadoDF.evento_id,"left") \
        .join(deportesDF,deportesOlimpicosDF.deporte_id == deportesDF.deporte_id,"left") \
        .select("sigla",
                "anio",
                "medalla",
                deportesOlimpicosDF.nombre.alias("Nombre subdisciplina"),
                deportesDF.deporte.alias("Nombre disciplina"),
                deportistaOlimpicoDF.nombre,   
                )

In [88]:
medallistaXAnio.show()

+-----+-------------+-------+--------------------+-----------------+--------------------+
|sigla|         anio|medalla|Nombre subdisciplina|Nombre disciplina|              nombre|
+-----+-------------+-------+--------------------+-----------------+--------------------+
|  BRU|  2016 Verano|     NA|Athletics Women's...|        Athletics|Maizurah Abdul Rahim|
|  BRU|  2000 Verano|     NA|Shooting Men's Skeet|         Shooting|Jefri Kiko Bolkia...|
|  BRU|  1996 Verano|     NA|Shooting Men's Skeet|         Shooting|Jefri Kiko Bolkia...|
|  BRU|  2004 Verano|     NA|Athletics Men's 1...|        Athletics|     Jimmy Anak Ahar|
|  BRU|  2000 Verano|     NA|Athletics Men's 1...|        Athletics|         Haseri Asli|
|  BRU|  2016 Verano|     NA|Athletics Men's 1...|        Athletics|Mohamed Fakhri Is...|
|  BRU|  2012 Verano|     NA|Swimming Men's 20...|         Swimming|Anderson Chee Wei...|
|  BRU|  2012 Verano|     NA|Athletics Women's...|        Athletics|      Maziah Mahusin|
|  BRU|  2

In [92]:
medallistaXAnio.where( col("Nombre subdisciplina").like("Alpine Skiing Wo%")) \
    .sort("anio") \
    .groupBy("Sigla","anio").count() \
    .show()

+-----+-------------+-----+
|Sigla|         anio|count|
+-----+-------------+-----+
|  SUI|1936 Invierno|    2|
|  TCH|1936 Invierno|    3|
|  ITA|1936 Invierno|    4|
|  NOR|1936 Invierno|    3|
|  USA|1936 Invierno|    4|
|  EST|1936 Invierno|    1|
|  ESP|1936 Invierno|    2|
|  GBR|1936 Invierno|    4|
|  AUT|1936 Invierno|    4|
|  LAT|1936 Invierno|    1|
|  CAN|1936 Invierno|    4|
|  GER|1936 Invierno|    4|
|  NED|1936 Invierno|    1|
|  FRA|1948 Invierno|   14|
|  SUI|1948 Invierno|   14|
|  TCH|1948 Invierno|    6|
|  ITA|1948 Invierno|    6|
|  NOR|1948 Invierno|    6|
|  USA|1948 Invierno|   14|
|  HUN|1948 Invierno|    3|
+-----+-------------+-----+
only showing top 20 rows



In [94]:
medallistaXAnio2 = medallistaXAnio.filter(medallistaXAnio.medalla != "NA") \
    .sort("anio") \
    .groupBy("Sigla","anio","Nombre subdisciplina") \
    .count()

In [95]:
medallistaXAnio2.printSchema()

root
 |-- Sigla: string (nullable = true)
 |-- anio: string (nullable = true)
 |-- Nombre subdisciplina: string (nullable = true)
 |-- count: long (nullable = false)



In [97]:
medallistaXAnio2.groupBy("Sigla","anio") \
    .agg( sum("count").alias("Total de medallas"), \
         avg("count").alias("Medallas promedio")) \
    .show()

+-----+-----------+-----------------+------------------+
|Sigla|       anio|Total de medallas| Medallas promedio|
+-----+-----------+-----------------+------------------+
|  FRA|1896 Verano|               11|             1.375|
|  SUI|1896 Verano|                3|               1.0|
|  GRE|1896 Verano|               48|1.6551724137931034|
|  USA|1896 Verano|               20|1.6666666666666667|
|  DEN|1896 Verano|                6|               1.0|
|  HUN|1896 Verano|                6|               1.0|
|  GBR|1896 Verano|                9|             1.125|
|  AUT|1896 Verano|                5|               1.0|
|  AUS|1896 Verano|                3|               1.0|
|  GER|1896 Verano|               32|2.6666666666666665|
|  FRA|1900 Verano|              250| 4.310344827586207|
|  GER|1900 Verano|               45| 6.428571428571429|
|  SUI|1900 Verano|               21|               3.5|
|  LUX|1900 Verano|                1|               1.0|
|  NED|1900 Verano|            

In [ ]:
#######Funciones escalares

In [109]:
#######SQL
resultadoDF.filter(resultadoDF.medalla != "NA") \
    .join(deportistaOlimpicoDF,deportistaOlimpicoDF.deportista_id == resultadoDF.deportista_id,"left") \
    .join(paisesDF,paisesDF.id == deportistaOlimpicoDF.equipo_id, "left") \
    .select("medalla", "equipo","sigla").sort( col("sigla").desc() ).show()

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



In [98]:
resultadoDF.registerTempTable("resultado")
deportistaOlimpicoDF.registerTempTable("deportista")
paisesDF.registerTempTable("paises")

In [104]:
sqlContext.sql("SELECT * FROM deportista").show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [108]:
sqlContext.sql("""
                SELECT medalla, equipo, sigla FROM resultado r
                JOIN deportista d
                ON r.deportista_id = d.deportista_id
                JOIN paises p
                ON p.id = d.equipo_id
                WHERE medalla <> "NA"
                ORDER BY sigla DESC
                """).show()

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



In [107]:
spark

<SparkContext master=local appName=clase14>

In [ ]:
#######SQL

In [115]:
###Persistencia
from pyspark.storagelevel import StorageLevel
medallistaXAnio.is_cached

False

In [122]:
medallistaXAnio.rdd.cache()

MapPartitionsRDD[493] at javaToPython at NativeMethodAccessorImpl.java:0

In [123]:
medallistaXAnio.rdd.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [125]:
medallistaXAnio.rdd.unpersist()

MapPartitionsRDD[493] at javaToPython at NativeMethodAccessorImpl.java:0

In [126]:
medallistaXAnio.rdd.persist(StorageLevel.MEMORY_AND_DISK_2)

MapPartitionsRDD[493] at javaToPython at NativeMethodAccessorImpl.java:0

In [127]:
medallistaXAnio.rdd.getStorageLevel()

StorageLevel(True, True, False, False, 2)

In [118]:
medallistaXAnio.rdd.getStorageLevel()

StorageLevel(True, True, False, False, 2)

In [129]:
#def __init__(self, useDisk, useMemory, useOffHeap, deserialized, replication=1):
StorageLevel.MEMORY_AND_DISK_3 = StorageLevel(True,True,False,False,3)

In [130]:
medallistaXAnio.rdd.unpersist()
medallistaXAnio.rdd.persist(StorageLevel.MEMORY_AND_DISK_3)

MapPartitionsRDD[493] at javaToPython at NativeMethodAccessorImpl.java:0